In [9]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3 as sql
import nltk
import string

#from sklearn.feature_extraction.text import TfidTransformer
#from sklearn.feature_extraction.text import TfidVectorizer
from sklearn.feature_extraction.text import CountVectorizer

con = sql.connect('database.sqlite')

In [10]:
filtered_data = pd.read_sql_query("""select * from reviews where score!=3 limit 10000""",con)


def partition(x):
    if x<3:
        return 0
    return 1

#changing the score less than 3 as negative and greater than 3 as positive
actual_score = filtered_data['Score']
positiveNegative = actual_score.map(partition)#here new column positivenegative has been created on the top of score
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (10000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [11]:
#this query is to remove the duplicates reviews done by the same usr at the same time
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", con)


print(display.shape)


display['COUNT(*)'].sum()


display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)

display.head(10)



#sorting the data as in ascending order

sorted_data = filtered_data.sort_values('ProductId',axis = 0,ascending =True,inplace=False,kind ='quicksort',na_position='last')

#deduplication of data
#subset of the function will the desired column that we want and keep is used 

final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)

final.shape




#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100





display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)

display.head()



final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]


#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(80668, 7)
(9564, 10)


1    7976
0    1588
Name: Score, dtype: int64

In [12]:
countvect = CountVectorizer()
final_counts = countvect.fit_transform(final['Text'].values)

In [13]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [15]:
final_counts.get_shape()

(9564, 18864)

In [ ]:
#here the value 18864 is the dimensionality of the total unique words that are present in the reviews column

